In [2]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Qdrant server URL
URL ="localhost"
# Qdrant dimension of the collection
DIMENSION = 384
# Qdrant collection name
COLLECTION_NAME = "data" #"incident data"
METRIC_NAME ="COSINE"

In [4]:
client = QdrantClient(URL,port=6333)

In [6]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [10]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core  import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

llm = OpenAI(api_base="http://localhost:1234/v1",
             api_key="lm-studio",
            #  model = 'lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf',
             )
Settings.embed_model = HuggingFaceEmbedding(
    model_name="nomic-ai/nomic-embed-text-v1.5",
    trust_remote_code=True
)
Settings.llm = llm


<All keys matched successfully>


In [11]:
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [12]:
query_engine = index.as_query_engine()

In [13]:
query = "What should government do about road safety?"
query = "What is the foreign participation in highways?"

In [14]:
response = query_engine.query(query)

UnexpectedResponse: Unexpected Response: 404 (Not Found)
Raw response content:
b'{"status":{"error":"Not found: Collection `data` doesn\'t exist!"},"time":0.000227584}'

In [1]:
pprint(response.get_formatted_sources())

Pretty printing has been turned OFF


In [29]:

print("query was:", query)
print("answer was:", response)

> Source (Doc id: 888a8a07-aa16-42ae-8d8f-71249e220188): 14  Report of the Committee
of motorization suggest that strategies for
reducing traffic injurie...

> Source (Doc id: 4dd6886a-6874-4f91-9c62-1bb03fbecba3): 8  Report of the Committee
road safety. There is also no effective
mechanism for coordinating th...
query was: What should government do about road safety?
answer was: Governments should make road safety a political priority, develop a multidisciplinary approach to road safety, appoint a lead agency for road safety, give it adequate resources, and make it publicly accountable. They should also set appropriate road safety targets and establish national road safety plans to achieve them, support the creation of safety advocacy groups, create budgets for road safety, and increase investment in demonstrably effective road safety activities.


In [41]:
from pprint import pprint

In [42]:
pprint(str(response))

('According to the excerpt from "Opportunities and Policy Initiatives for '
 'Profitable Partnership and Infrastructure Investment in Indian Highways and '
 'National Highways Development Project: A Comprehensive Overview," '
 'contractors and supervision consultants from around 27 countries are already '
 "in India, helping implement one of the world's largest highways projects. "
 'The excerpt also provides a table listing the countries with contractors and '
 "consultants currently working on India's highways projects. Therefore, "
 'foreign participation in highways in India is significant.')


In [43]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

In [44]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
    llm = llm,
)

In [45]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

In [46]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query(query)

In [47]:
pprint(str(response))

('According to the excerpt from "Opportunities and Policy Initiatives for '
 'Profitable Partnership and Infrastructure Investment in Indian Highways and '
 'National Highways Development Project: A Comprehensive Overview," '
 'contractors and supervision consultants from around 27 countries are already '
 "in India, helping implement one of the world's largest highways projects. "
 'The excerpt also provides a table that lists the number of contractors and '
 "consultants from each country that are currently working on India's highways "
 'projects. Therefore, foreign participation in highways in India is '
 'significant.')
